In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import numpy as np

from torch.utils.data import TensorDataset, DataLoader
import lightning as L
import mlflow

In [3]:


# Data has already been prepared for you
train = np.load('HA3_train_set.npz')
test = np.load('HA3_test_set.npz')
X_train = train['x_train']
y_train = train['y_train']
X_test = test['x_test']
y_test = test['y_test']

# Step 1: Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


# Step 2: Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [4]:
print("X_train.shape:", X_train.shape)
print("y_train.shape:", y_train.shape)
print("X_test.shape:", X_test.shape)
print("y_test.shape:", y_test.shape)

X_train.shape: (25000, 500)
y_train.shape: (25000,)
X_test.shape: (17500, 500)
y_test.shape: (17500,)


In [10]:
# Recurrent neural network (many-to-one)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.softmax = nn.LogSoftmax(dim = 1)

    def forward(self, x):
        # Forward propagate LSTM
        out, _ = self.lstm(x)  # out: tensor of shape (batch_size, seq_length, hidden_size)
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        out = self.softmax(out)
        return out



In [11]:
def train_model(model, input_size, num_epochs, learning_rate, train_loader, test_loader):
    # Loss and optimizer
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    best_accuracy = 0.0
    global_step = 0

    # Train the model
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        for i, (comments, labels) in enumerate(train_loader):
            comments = comments.view(comments.shape[0], -1, input_size) 
            global_step += 1

            # Forward pass
            outputs = model(comments)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            mlflow.log_metric("loss", loss.item(), step = global_step)

            if (i+1) % 10 == 0:
                print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
                
                # Test the model
                with torch.no_grad():
                    correct = 0
                    total = 0
                    for comments, labels in test_loader:
                        comments = comments.view(comments.shape[0], -1, input_size) 
                        outputs = model(comments)
                        _, predicted = torch.max(outputs.data, 1)
                        total += labels.size(0)
                        correct += (predicted == labels).sum().item()
                        accuracy = 100 * correct / total
                    mlflow.log_metric("Test accuracy vs number of training epochs", 100 * correct / total, step = global_step)

                # Save the best model
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    mlflow.pytorch.log_model(pytorch_model=model, artifact_path="best-torch-rnn-model")

                print('Test Accuracy of the model on test comments: {} %'.format(100 * correct / total))
                
    return best_accuracy


In [18]:
import optuna

def objective(trial):
    
    num_layers = 2
    hidden_size = 1
    batch_size = 2500
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01)
    input_size = 20


    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

    num_classes = 2  # Fixed number of classes
    num_epochs = 10  # For faster trials, you might want to use more epochs in actual training

    with mlflow.start_run():
        mlflow.log_param("num_hidden_layers", num_layers)
        mlflow.log_param("hidden_size", hidden_size)
        mlflow.log_param("batch_size", batch_size)
        mlflow.log_param("epochs", num_epochs)
        mlflow.log_param("learning_rate", learning_rate)
        mlflow.log_param("input_size", input_size)

        model = RNN(input_size, hidden_size, num_layers, num_classes)
        print('----------------------------------------------------------')
        best_accuracy = train_model(model, input_size, num_epochs, learning_rate, train_loader, test_loader)  # Modify train_model to return accuracy

        mlflow.log_metric("best_accuracy", best_accuracy)

    return best_accuracy


In [19]:
# Create a study and optimize the objective function
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=3)

# Log the best hyperparameters
best_trial = study.best_trial
print(f'Best trial: {best_trial.value}')
print(f'Best hyperparameters: {best_trial.params}')



[I 2024-06-04 17:02:25,000] A new study created in memory with name: no-name-11aac609-df86-42c0-8336-86a59b9616c0


----------------------------------------------------------
Epoch [1/10], Step [10/10], Loss: 0.7129
Test Accuracy of the model on test comments: 50.04571428571428 %
Epoch [2/10], Step [10/10], Loss: 0.6940
Test Accuracy of the model on test comments: 50.04571428571428 %
Epoch [3/10], Step [10/10], Loss: 0.6946
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [4/10], Step [10/10], Loss: 0.6934
Test Accuracy of the model on test comments: 49.94285714285714 %
Epoch [5/10], Step [10/10], Loss: 0.6929
Test Accuracy of the model on test comments: 50.12 %
Epoch [6/10], Step [10/10], Loss: 0.6931
Test Accuracy of the model on test comments: 50.28 %
Epoch [7/10], Step [10/10], Loss: 0.6930
Test Accuracy of the model on test comments: 50.31428571428572 %
Epoch [8/10], Step [10/10], Loss: 0.6931
Test Accuracy of the model on test comments: 50.19428571428571 %
Epoch [9/10], Step [10/10], Loss: 0.6929
Test Accuracy of the model on test comments: 50.205714285714286 %


[I 2024-06-04 17:02:51,898] Trial 0 finished with value: 50.31428571428572 and parameters: {'learning_rate': 0.009376869555272914}. Best is trial 0 with value: 50.31428571428572.


Epoch [10/10], Step [10/10], Loss: 0.6931
Test Accuracy of the model on test comments: 50.177142857142854 %
----------------------------------------------------------
Epoch [1/10], Step [10/10], Loss: 0.7884
Test Accuracy of the model on test comments: 50.04571428571428 %
Epoch [2/10], Step [10/10], Loss: 0.7342
Test Accuracy of the model on test comments: 50.04571428571428 %
Epoch [3/10], Step [10/10], Loss: 0.7129
Test Accuracy of the model on test comments: 49.88 %
Epoch [4/10], Step [10/10], Loss: 0.7001
Test Accuracy of the model on test comments: 49.68571428571428 %
Epoch [5/10], Step [10/10], Loss: 0.6983
Test Accuracy of the model on test comments: 49.8 %
Epoch [6/10], Step [10/10], Loss: 0.7004
Test Accuracy of the model on test comments: 49.78285714285714 %
Epoch [7/10], Step [10/10], Loss: 0.6975
Test Accuracy of the model on test comments: 49.80571428571429 %
Epoch [8/10], Step [10/10], Loss: 0.6956
Test Accuracy of the model on test comments: 49.89714285714286 %
Epoch [9/1

[I 2024-06-04 17:03:08,492] Trial 1 finished with value: 50.04571428571428 and parameters: {'learning_rate': 0.006789516238133524}. Best is trial 0 with value: 50.31428571428572.


Test Accuracy of the model on test comments: 49.64 %
----------------------------------------------------------
Epoch [1/10], Step [10/10], Loss: 0.8365
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [2/10], Step [10/10], Loss: 0.8321
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [3/10], Step [10/10], Loss: 0.7775
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [4/10], Step [10/10], Loss: 0.7692
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [5/10], Step [10/10], Loss: 0.7731
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [6/10], Step [10/10], Loss: 0.7588
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [7/10], Step [10/10], Loss: 0.7398
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [8/10], Step [10/10], Loss: 0.7232
Test Accuracy of the model on test comments: 49.95428571428572 %
Epoch [9/10], Step [10/10], Loss: 0.7184

[I 2024-06-04 17:03:26,066] Trial 2 finished with value: 49.95428571428572 and parameters: {'learning_rate': 0.004163104130769196}. Best is trial 0 with value: 50.31428571428572.


Test Accuracy of the model on test comments: 49.95428571428572 %
Best trial: 50.31428571428572
Best hyperparameters: {'learning_rate': 0.009376869555272914}
